# Module 1: Serving fresh online features with Feast, Kafka, Redis

## 1. Overview
In this notebook, we explore using Spark to build streaming features from events in Kafka and registering them within Feast. We then showcase how Feast combines these streaming features with batch data sources in the online store (Redis). Users can then retrieve features at low latency from Redis through Feast.

If you haven't already, look at the [README](../README.md) for setup instructions prior to starting this notebook.

<img src="../architecture.png" width="750"/>

## 2. Setup Spark Structured Streaming to read this Kafka Topic
We first read and parse events from Kafka, run some transformations with Spark, and `forEachBatch` push to Feast

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, IntegerType, DoubleType, TimestampType

import pandas as pd
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell"

In [2]:
spark = SparkSession.builder.master("local").appName("feast-spark").getOrCreate()
# Reduce partitions since default is 200 which will be slow on a local machine
spark.conf.set("spark.sql.shuffle.partitions", 5)

schema = (
    StructType()
        .add('driver_id', IntegerType(), False)
        .add('miles_driven', DoubleType(), False)
        .add('event_timestamp', TimestampType(), False)
        .add('conv_rate', DoubleType(), False)
        .add('acc_rate', DoubleType(), False)
)

# Subscribe to 1 topic, with headers
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "drivers")
    .option("startingOffsets", "earliest")
    .load()
    .selectExpr('CAST(value AS STRING)')
    .select(from_json('value', schema).alias("temp"))
    .select("temp.*")
)

:: loading settings :: url = jar:file:/Users/dannychiao/.pyenv/versions/3.8.10/envs/python-3.8-test/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dannychiao/.ivy2/cache
The jars for the packages stored in: /Users/dannychiao/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dc6fba36-cdf6-4978-85e5-d5d45ac232a6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 539ms :: artifacts dl 24ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.4-3 from central in [default]
	org.apache.commons#commons-

# 3. Setup the feature store

### Apply feature repository
We first run `feast apply` to register the data sources + features and setup Redis.

In [3]:
!feast apply

Created entity driver
Created feature view driver_hourly_stats
Created feature view driver_daily_features
Created feature service model_v2

Deploying infrastructure for driver_hourly_stats
Deploying infrastructure for driver_daily_features


Now, we instantiate a Feast `FeatureStore` object to push data to

In [4]:
from feast import FeatureStore
from datetime import datetime

store = FeatureStore(repo_path=".")

### Fetch training data from offline store
Just to verify the features are in the batch sources.

In [5]:

entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003, 1004, 1001],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
            datetime(2021, 4, 12, 15, 1, 12),
            datetime.now()
        ]
    }
)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_daily_features:daily_miles_driven"
    ],
).to_df()
print(training_df.head())

      driver_id                  event_timestamp  conv_rate  acc_rate  \
360        1001        2021-04-12 10:59:42+00:00   0.521149  0.751659   
721        1002        2021-04-12 08:12:10+00:00   0.089014  0.212637   
1084       1003        2021-04-12 16:40:26+00:00   0.188855  0.344736   
1445       1004        2021-04-12 15:01:12+00:00   0.296492  0.935305   
1805       1001 2022-05-16 00:58:22.547675+00:00   0.404588  0.407571   

      daily_miles_driven  
360            18.926695  
721            12.005569  
1084           23.490234  
1445           19.204191  
1805          350.650257  


### 4. Materialize batch features & fetch online features from Redis
First we materialize features (which generate the latest values for each entity key from batch sources) into the online store (Redis)

In [6]:
!feast materialize-incremental $(date +%Y-%m-%d)

Materializing 2 feature views to 2022-05-15 20:00:00-04:00 into the redis online store.

driver_hourly_stats from 1748-07-31 04:58:27-04:56:02 to 2022-05-15 20:00:00-04:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 331.12it/s]
driver_daily_features from 1748-07-31 04:58:27-04:56:02 to 2022-05-15 20:00:00-04:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 908.41it/s]


Feast manages what time intervals have been materialized in the registry. So if you schedule regular materialization every hour, you can run `feast materialize-incremental` and Feast will know that all the previous hours were already processed.

#### SDK based online retrieval
Now we can retrieve these materialized features from Redis by directly using the SDK. This is one of the most popular ways to retrieve features with Feast since it allows you to integrate with an existing service (e.g. a Flask) that also handles model inference or pre/post-processing

In [7]:
features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_daily_features:daily_miles_driven",
    ],
    entity_rows=[
        {
            "driver_id": 1001,
        }
    ],
).to_dict()

def print_online_features(features):
    for key, value in sorted(features.items()):
        print(key, " : ", value)

print_online_features(features)

acc_rate  :  [0.4075707495212555]
conv_rate  :  [0.4045884609222412]
daily_miles_driven  :  [350.6502685546875]
driver_id  :  [1001]


#### HTTP based online retrieval
We can also retrieve from a deployed feature server. We had previously deployed this with Docker Compose (see [docker-compose.yml](../docker-compose.yml))

This can be preferable for many reasons. If you want to build an in-memory cache, caching on a central feature server can allow more effective caching across teams. You can also more centrally manage rate-limiting / access control, upgrade Feast versions independently, etc.

In [8]:
import requests
import json

online_request = {
  "features": [
    "driver_hourly_stats:conv_rate",
    "driver_hourly_stats:acc_rate"
  ],
  "entities": {
    "driver_id": [1001]
  }
}
r = requests.post('http://localhost:6566/get-online-features', data=json.dumps(online_request))
print(json.dumps(r.json(), indent=4, sort_keys=True))


{
    "metadata": {
        "feature_names": [
            "driver_id",
            "conv_rate",
            "acc_rate"
        ]
    },
    "results": [
        {
            "event_timestamps": [
                "1970-01-01T00:00:00Z"
            ],
            "statuses": [
                "PRESENT"
            ],
            "values": [
                1001
            ]
        },
        {
            "event_timestamps": [
                "2022-03-14T14:00:00Z"
            ],
            "statuses": [
                "PRESENT"
            ],
            "values": [
                0.4045884609222412
            ]
        },
        {
            "event_timestamps": [
                "2022-03-14T14:00:00Z"
            ],
            "statuses": [
                "PRESENT"
            ],
            "values": [
                0.4075707495212555
            ]
        }
    ]
}


## 5. Generating fresher features via stream transformations

### 5a. Building streaming features with Kafka + Spark Structured Streaming
Now we push streaming features into Feast by ingesting events from Kafka and processing with Spark Structured Streaming.
- These features can then be further post-processed and combined with other features or request data in on demand transforms.
- An example might be to push in the last 5 transactions, and in on demand transforms generate the average of those transactions.

Feast will help manage both batch and streaming sources for you. You can run `feast materialize-incremental` as well as ingest streaming features to the same online store.

In [9]:
def send_to_feast(df, epoch):
    pandas_df: pd.DataFrame = df.toPandas()
    if pandas_df.empty:
        return
    
    if "end" in pandas_df:
        print("processing window")
        # Filter out only for the latest window for the driver id
        pandas_df = pandas_df.sort_values(by=["driver_id","end"], ascending=False).groupby("driver_id").nth(-1)
        pandas_df = pandas_df.rename(columns = {"end": "event_timestamp"})
        pandas_df['created'] = pd.to_datetime('now', utc=True)
        store.push("driver_stats_push_source", pandas_df)
    pandas_df.sort_values(by="driver_id", inplace=True)
    print(pandas_df.head(20))
    print(f"Num rows: {len(pandas_df.index)}")

daily_miles_driven = (
    df.withWatermark("event_timestamp", "1 second") 
        .groupBy("driver_id", window(timeColumn="event_timestamp", windowDuration="1 day", slideDuration="1 hour"))
        .agg(sum("miles_driven").alias("daily_miles_driven"))
        .select("driver_id", "window.end", "daily_miles_driven")
)

query_1 = daily_miles_driven \
    .writeStream \
    .outputMode("update") \
    .option("checkpointLocation", "/tmp/feast-workshop/q1/") \
    .trigger(processingTime="15 seconds") \
    .foreachBatch(send_to_feast) \
    .start()

query_1.awaitTermination(timeout=15)
query_1.stop()

22/05/16 00:58:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


processing window
              event_timestamp  daily_miles_driven  \
driver_id                                           
1001      2022-04-11 08:00:00           18.926695   
1002      2022-04-11 08:00:00           12.005569   
1003      2022-04-11 08:00:00           23.490234   
1004      2022-04-11 08:00:00           19.204191   
1005      2022-04-11 08:00:00            5.764504   

                                   created  
driver_id                                   
1001      2022-05-16 04:58:51.192354+00:00  
1002      2022-05-16 04:58:51.192354+00:00  
1003      2022-05-16 04:58:51.192354+00:00  
1004      2022-05-16 04:58:51.192354+00:00  
1005      2022-05-16 04:58:51.192354+00:00  
Num rows: 5


/Users/dannychiao/.pyenv/versions/3.8.10/envs/python-3.8-test/lib/python3.8/site-packages/feast/feature_store.py:1219: RuntimeWarning: Push source is an experimental feature. This API is unstable and it could and might change in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


processing window
              event_timestamp  daily_miles_driven  \
driver_id                                           
1001      2023-03-01 06:00:00          588.616454   
1002      2023-03-01 07:00:00          675.603021   
1003      2023-03-01 07:00:00          542.164027   
1004      2023-03-01 06:00:00          624.280938   
1005      2023-03-01 06:00:00          600.712434   

                                   created  
driver_id                                   
1001      2022-05-16 04:58:52.500009+00:00  
1002      2022-05-16 04:58:52.500009+00:00  
1003      2022-05-16 04:58:52.500009+00:00  
1004      2022-05-16 04:58:52.500009+00:00  
1005      2022-05-16 04:58:52.500009+00:00  
Num rows: 5


/Users/dannychiao/.pyenv/versions/3.8.10/envs/python-3.8-test/lib/python3.8/site-packages/feast/feature_store.py:1219: RuntimeWarning: Push source is an experimental feature. This API is unstable and it could and might change in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


#### 5b. Verify fresh features
Now we can verify that the `daily_miles_driven` feature has indeed changed from the original materialized features.

In [10]:
features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_daily_features:daily_miles_driven",
    ],
    entity_rows=[
        {
            "driver_id": 1001,
        }
    ],
).to_dict()
print_online_features(features)

acc_rate  :  [0.4075707495212555]
conv_rate  :  [0.4045884609222412]
daily_miles_driven  :  [588.616455078125]
driver_id  :  [1001]


#### 5c (optional): Push features through the push server
We could also have pushed features without a `FeatureStore` object (and hence the Spark jobs don't need a `feature_store.yaml`) and instead through an HTTP request to the push server (`requests.post("http://localhost:6567/push", data=json.dumps(push_data))`) 

We already deployed a push server with Docker Compose at port 6567. Separating writes (pushes) from reads (feature serving) can be helpful for independent scaling + resource isolation.

In [11]:
import requests
import json

def send_to_feast(df, epoch):
    pandas_df: pd.DataFrame = df.toPandas()
    if pandas_df.empty:
        return
    
    if "end" in pandas_df:
        print("processing window")
        # Filter out only for the latest window for the driver id
        pandas_df = pandas_df.sort_values(by=["driver_id","end"], ascending=False).groupby("driver_id").nth(-1)
        pandas_df = pandas_df.rename(columns = {"end": "event_timestamp"})
        pandas_df['created'] = pd.to_datetime('now', utc=True)
        pandas_df.reset_index(inplace=True)

        # Push the event via HTTP (we need to map timestamps to strings since timestamps aren't serializable)
        pandas_df['created'] = pandas_df['created'].astype(str)
        pandas_df['event_timestamp'] = pandas_df['event_timestamp'].astype(str)
        pandas_dict = pandas_df.to_dict(orient="list")
        push_data = {
            "push_source_name":"driver_stats_push_source",
            "df":pandas_dict
        }
        requests.post("http://localhost:6567/push", data=json.dumps(push_data))
    pandas_df.sort_values(by="driver_id", inplace=True)
    print(pandas_df.head(20))
    print(f"Num rows: {len(pandas_df.index)}")

query_2 = daily_miles_driven \
    .writeStream \
    .outputMode("update") \
    .option("checkpointLocation", "/tmp/feast-workshop/q2/") \
    .trigger(processingTime="15 seconds") \
    .foreachBatch(send_to_feast) \
    .start()

query_2.awaitTermination(timeout=10)
query_2.stop()

22/05/16 00:59:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


processing window
   driver_id      event_timestamp  daily_miles_driven  \
0       1001  2022-04-11 08:00:00           18.926695   
1       1002  2022-04-11 08:00:00           12.005569   
2       1003  2022-04-11 08:00:00           23.490234   
3       1004  2022-04-11 08:00:00           19.204191   
4       1005  2022-04-11 08:00:00            5.764504   

                            created  
0  2022-05-16 04:59:12.508793+00:00  
1  2022-05-16 04:59:12.508793+00:00  
2  2022-05-16 04:59:12.508793+00:00  
3  2022-05-16 04:59:12.508793+00:00  
4  2022-05-16 04:59:12.508793+00:00  
Num rows: 5
processing window
   driver_id      event_timestamp  daily_miles_driven  \
0       1001  2023-03-03 07:00:00          565.543522   
1       1002  2023-03-03 08:00:00          639.230220   
2       1003  2023-03-03 08:00:00          482.580150   
3       1004  2023-03-03 07:00:00          610.192632   
4       1005  2023-03-03 07:00:00          496.704214   

                            created  


Now we can verify yet again that features are more fresh

In [12]:
features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_daily_features:daily_miles_driven",
    ],
    entity_rows=[
        {
            "driver_id": 1001,
        }
    ],
).to_dict()
print_online_features(features)

acc_rate  :  [0.4075707495212555]
conv_rate  :  [0.4045884609222412]
daily_miles_driven  :  [565.5435180664062]
driver_id  :  [1001]


### Cleanup
Finally, let's clean up the checkpoint directories from Spark

In [13]:
import shutil

dir_path = '/tmp/feast-workshop/'

try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))